<a href="https://colab.research.google.com/github/mancher07/samsung/blob/main/GradioLive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.38.2 tiktoken torch numpy gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [ ]:
import argparse
import os
import gradio as gr
import json
from threading import Thread
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [ ]:

MAX_LENGTH = 4096
DEFAULT_MAX_NEW_TOKENS = 1024

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b", torch_dtype=torch.bfloat16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def predict(message, history, system_prompt, temperature, max_tokens):
    instruction = system_prompt
    for human, assistant in history:
        instruction += f'user\n{human}\n\nassistant\n{assistant}\n'
    instruction += f'\nuser\n{message}\n\nassistant\n'

    problem = [instruction]
    stop_tokens = ["", ""]
    streamer = TextIteratorStreamer(tokenizer, timeout=100.0, skip_prompt=True, skip_special_tokens=True)
    enc = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask

    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, -MAX_LENGTH:]

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    generate_kwargs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "streamer": streamer,
        "do_sample": True,
        "top_p": 0.95,
        "temperature": temperature,
        "max_new_tokens": max_tokens,
    }
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        if text in stop_tokens:
            break
        yield "".join(outputs)

In [3]:

import argparse
import os
import gradio as gr
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

# Variables globales y configuración del modelo
MAX_LENGTH = 4096
DEFAULT_MAX_NEW_TOKENS = 1024

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b", torch_dtype=torch.bfloat16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def predict(message, history, system_prompt, temperature, max_tokens):
    instruction = system_prompt
    for human, assistant in history:
        instruction += f'user\n{human}\n\nassistant\n{assistant}\n'
    instruction += f'\nuser\n{message}\n\nassistant\n'

    problem = [instruction]
    streamer = TextIteratorStreamer(tokenizer, timeout=100.0, skip_prompt=True, skip_special_tokens=True)
    enc = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask

    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, -MAX_LENGTH:]

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    generate_kwargs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "streamer": streamer,
        "do_sample": True,
        "top_p": 0.95,
        "temperature": temperature,
        "max_new_tokens": max_tokens,
    }

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
        yield "".join(outputs)
    yield "".join(outputs)

system_prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."

interface = gr.ChatInterface(
    fn=predict,
    title="Welcome to Gradio",
    description="Chat Model Stable Code 3B",
    theme="Glass",
    chatbot=gr.Chatbot(label="Chat History"),
    textbox=gr.Textbox(placeholder="input", container=False, scale=7),
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
    additional_inputs=[
        gr.Textbox(system_prompt, label="System Prompt"),
        gr.Slider(0, 1, 0.9, label="Temperature"),
        gr.Slider(100, 2048, 1024, label="Max Tokens"),
    ],
    additional_inputs_accordion="Parameters",
)

interface.launch(share=None)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://db95825b09e94c0b98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

# Variables globales y configuración del modelo
MAX_LENGTH = 4096
DEFAULT_MAX_NEW_TOKENS = 1024

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b", torch_dtype=torch.bfloat16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Widgets para la interfaz de usuario
title_input = widgets.Text(value='Welcome to Gradio', description='Title:')
system_prompt_input = widgets.Textarea(value="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", description='System Prompt:', layout=widgets.Layout(width='80%'))
temperature_slider = widgets.FloatSlider(value=0.9, min=0, max=1, step=0.01, description='Temperature:')
max_tokens_slider = widgets.IntSlider(value=1024, min=100, max=2048, step=1, description='Max Tokens:')
message_input = widgets.Textarea(value='Hello!', description='Message:', layout=widgets.Layout(width='80%'))
execute_button = widgets.Button(description='Execute')

# Función de predicción
def predict(message, system_prompt, temperature, max_tokens):
    instruction = system_prompt + f'\nuser\n{message}\n\nassistant\n'
    problem = [instruction]
    streamer = TextIteratorStreamer(tokenizer, timeout=100.0, skip_prompt=True, skip_special_tokens=True)
    enc = tokenizer(problem, return_tensors="pt", padding=True, truncation=True)
    input_ids = enc.input_ids.to(device)
    attention_mask = enc.attention_mask.to(device)

    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, -MAX_LENGTH:]

    generate_kwargs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask.to(device),
        "streamer": streamer,
        "do_sample": True,
        "top_p": 0.95,
        "temperature": temperature,
        "max_new_tokens": max_tokens,
    }

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    outputs = []
    for text in streamer:
        outputs.append(text)
    return "".join(outputs)

# Función para ejecutar el modelo con los parámetros proporcionados
def on_execute_button_clicked(b):
    clear_output()
    display(widgets.VBox([title_input, system_prompt_input, temperature_slider, max_tokens_slider, message_input, execute_button]))
    output = predict(message_input.value, system_prompt_input.value, temperature_slider.value, max_tokens_slider.value)
    print("Output:", output)

execute_button.on_click(on_execute_button_clicked)

# Mostrar la interfaz de usuario
display(widgets.VBox([title_input, system_prompt_input, temperature_slider, max_tokens_slider, message_input, execute_button]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
